# Data labeling example
This notebook shows how to use EDSL to automate data labeling and content analysis. Using a dataset of mock customer service tickets as an example, we design a data labeling task as a survey of questions about the tickets that we administer to a language model, generating a summary dataset.

The steps are:
1. Technical setup<br>
2. Identifying data for review<br>
3. Constructing questions about the data<br>
4. Delivering the questions to language models<br>
5. Analyzing the responses<br>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/data_labeling_example.ipynb)

### Technical setup
Before running the code below please see instructions on:

* [Installing the EDSL package](https://docs.expectedparrot.com/en/latest/installation.html) and
* [Storing API Keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use

We also have a [Starter Tutorial](https://docs.expectedparrot.com/en/latest/starter_tutorial.html) about EDSL basics.

In [1]:
# ! pip install edsl

## Selecting data for review
First we identify some data for review. Data can be created using the EDSL tools or imported from other sources. For purposes of this demo we import a set of hypothetical customer tickets for a transportation app:

In [2]:
tickets = [
    "I just realized I left my phone in the car on my last ride. Can you help me get it back?",
    "I'm unhappy with my recent experience. The driver was very rude and unprofessional.",
    "I was charged more than the estimated fare for my trip yesterday. Can you explain why?",
    "The car seat provided was not properly installed, and I felt my child was at risk. Please ensure driver training.",
    "My driver took a longer route than necessary, resulting in a higher fare. I request a fare adjustment.",
    "I had a great experience with my driver today! Very friendly and efficient service.",
    "I'm concerned about the vehicle's cleanliness. It was not up to the standard I expect.",
    "The app keeps crashing every time I try to book a ride. Please fix this issue.",
    "My driver was exceptional - safe driving, polite, and the car was spotless. Kudos!",
    "I felt unsafe during my ride due to the driver's erratic behavior. This needs to be addressed immediately.",
    "The driver refused to follow my preferred route, which is shorter. I'm not satisfied with the service.",
    "Impressed with the quick response to my ride request and the driver's professionalism.",
    "I was charged for a ride I never took. Please refund me as soon as possible.",
    "The promo code I tried to use didn't work. Can you assist with this?",
    "There was a suspicious smell in the car, and I'm worried about hygiene standards.",
    "My driver was very considerate, especially helping me with my luggage. Appreciate the great service!",
    "The app's GPS seems inaccurate. It directed the driver to the wrong pick-up location.",
    "I want to compliment my driver's excellent navigation and time management during rush hour.",
    "The vehicle didn't match the description in the app. It was confusing and concerning.",
    "I faced an issue with payment processing after my last ride. Can you look into this?",
]

## Constructing questions about the data
Next we create some questions about the data. EDSL provides classes for a variety of common question types, including multiple choice, free text, checkbox, linear scale and many others. We import the question type classes that we want to use and then construct questions in the relevant templates. We use a <b><i>{{ placeholder }}</b></i> in each question text in order to parameterize the questions with the individual ticket contents. For more details about constructing questions, please see the <a href="https://docs.expectedparrot.com/en/latest/questions.html">Questions</a> section of the docs.

In [3]:
from edsl.questions import QuestionMultipleChoice, QuestionCheckBox, QuestionFreeText, QuestionList, QuestionYesNo, QuestionLinearScale

In [4]:
question_issues = QuestionCheckBox(
    question_name = "issues",
    question_text = "Check all of the issues mentioned in this ticket: {{ ticket }}",
    question_options = [
        "safety", 
        "cleanliness", 
        "driver performance", 
        "GPS/route", 
        "lost item", 
        "other"]
)

In [5]:
question_primary_issue = QuestionFreeText(
    question_name = "primary_issue",
    question_text = "What is the primary issue in this ticket? Ticket: {{ ticket }}"
)

In [6]:
question_accident = QuestionMultipleChoice(
    question_name = "accident",
    question_text = "If the primary issue in this ticket is safety, was there an accident where someone was hurt? Ticket: {{ ticket }}",
    question_options = ["Yes", "No", "Not applicable"]
)

In [7]:
question_sentiment = QuestionMultipleChoice(
    question_name = "sentiment",
    question_text = "What is the sentiment of this ticket? Ticket: {{ ticket }}",
    question_options = [
        "Very positive",
        "Somewhat positive",
        "Neutral",
        "Somewhat negative",
        "Very negative"
    ]
)

In [8]:
question_refund = QuestionYesNo(
    question_name = "refund",
    question_text = "Does the customer ask for a refund in this ticket? Ticket: {{ ticket }}"
)

In [9]:
question_priority = QuestionLinearScale(
    question_name = "priority",
    question_text = "On a scale from 0 to 5, what is the priority level of this ticket? Ticket: {{ ticket }}",
    question_options = [0,1,2,3,4,5],
    option_labels = {0: "Lowest", 5: "Highest"}
)

## Building a survey
We combine the questions into a survey in order to administer them together:

In [10]:
from edsl import Survey

survey = Survey(
    questions = [
        question_issues,
        question_primary_issue,
        question_accident,
        question_sentiment,
        question_refund,
        question_priority
    ]
)

(Survey questions are administered asynchronously by default. To learn about adding conditional logic such as skip/stop rules and targeted memory, please see the <a href="https://docs.expectedparrot.com/en/latest/surveys.html">Surveys</a> section of the docs.)

We can review our questions in a readable format, or export them as a survey to use with human respondents or at other survey platforms:

In [11]:
survey

Survey(questions=[Question('checkbox', question_name = 'issues', question_text = 'Check all of the issues mentioned in this ticket: {{ ticket }}', min_selections = None, max_selections = None, question_options = ['safety', 'cleanliness', 'driver performance', 'GPS/route', 'lost item', 'other']), Question('free_text', question_name = 'primary_issue', question_text = 'What is the primary issue in this ticket? Ticket: {{ ticket }}'), Question('multiple_choice', question_name = 'accident', question_text = 'If the primary issue in this ticket is safety, was there an accident where someone was hurt? Ticket: {{ ticket }}', question_options = ['Yes', 'No', 'Not applicable']), Question('multiple_choice', question_name = 'sentiment', question_text = 'What is the sentiment of this ticket? Ticket: {{ ticket }}', question_options = ['Very positive', 'Somewhat positive', 'Neutral', 'Somewhat negative', 'Very negative']), Question('yes_no', question_name = 'refund', question_text = 'Does the customer ask for a refund in this ticket? Ticket: {{ ticket }}', question_options = ['Yes', 'No']), Question('linear_scale', question_name = 'priority', question_text = 'On a scale from 0 to 5, what is the priority level of this ticket? Ticket: {{ ticket }}', question_options = [0, 1, 2, 3, 4, 5], option_labels = {0: 'Lowest', 5: 'Highest'})], memory_plan={}, rule_collection=RuleCollection(rules=[Rule(current_q=0, expression="True", next_q=1, priority=-1, question_name_to_index={'issues': 0}, before_rule=False), Rule(current_q=1, expression="True", next_q=2, priority=-1, question_name_to_index={'issues': 0, 'primary_issue': 1}, before_rule=False), Rule(current_q=2, expression="True", next_q=3, priority=-1, question_name_to_index={'issues': 0, 'primary_issue': 1, 'accident': 2}, before_rule=False), Rule(current_q=3, expression="True", next_q=4, priority=-1, question_name_to_index={'issues': 0, 'primary_issue': 1, 'accident': 2, 'sentiment': 3}, before_rule=False), Rule(current_q=4, expression="True", next_q=5, priority=-1, question_name_to_index={'issues': 0, 'primary_issue': 1, 'accident': 2, 'sentiment': 3, 'refund': 4}, before_rule=False), Rule(current_q=5, expression="True", next_q=6, priority=-1, question_name_to_index={'issues': 0, 'primary_issue': 1, 'accident': 2, 'sentiment': 3, 'refund': 4, 'priority': 5}, before_rule=False)], num_questions=6), question_groups={})

## Designing AI agents
A key feature of EDSL is the ability to create personas for AI agents that the language models are prompted to use in generating responses to the questions. This is done by passing a dictionary of traits to Agent objects:

In [12]:
from edsl import Agent

agent = Agent(traits = {
    "persona": "You are an expert customer service agent.",
    "years_experience": 15
})

## Selecting language models
EDSL allows us to select the language models to use in generating results. To see all available models:

In [13]:
from edsl import Model

Model.available()

[['01-ai/Yi-34B-Chat', 'deep_infra', 0],
 ['Austism/chronos-hermes-13b-v2', 'deep_infra', 1],
 ['Gryphe/MythoMax-L2-13b', 'deep_infra', 2],
 ['Gryphe/MythoMax-L2-13b-turbo', 'deep_infra', 3],
 ['HuggingFaceH4/zephyr-orpo-141b-A35b-v0.1', 'deep_infra', 4],
 ['Phind/Phind-CodeLlama-34B-v2', 'deep_infra', 5],
 ['bigcode/starcoder2-15b', 'deep_infra', 6],
 ['bigcode/starcoder2-15b-instruct-v0.1', 'deep_infra', 7],
 ['claude-3-haiku-20240307', 'anthropic', 8],
 ['claude-3-opus-20240229', 'anthropic', 9],
 ['claude-3-sonnet-20240229', 'anthropic', 10],
 ['codellama/CodeLlama-34b-Instruct-hf', 'deep_infra', 11],
 ['codellama/CodeLlama-70b-Instruct-hf', 'deep_infra', 12],
 ['cognitivecomputations/dolphin-2.6-mixtral-8x7b', 'deep_infra', 13],
 ['databricks/dbrx-instruct', 'deep_infra', 14],
 ['deepinfra/airoboros-70b', 'deep_infra', 15],
 ['gemini-pro', 'google', 16],
 ['google/codegemma-7b-it', 'deep_infra', 17],
 ['google/gemma-1.1-7b-it', 'deep_infra', 18],
 ['gpt-3.5-turbo', 'openai', 19],


Here we select GPT 4 (which is also used by default if no model is specified for a survey):

In [14]:
model = Model("gpt-4-1106-preview")

## Conducting the analysis
With our data and questions we're now ready to package our survey and deliver it to an AI. We do this by inserting the contents of each ticket into each question as an independent "scenario" for review, and then running the survey.

In [15]:
from edsl import Scenario

scenarios = [Scenario({"ticket":t}) for t in tickets] 

We run the survey by chaining the components with the <i>by</i> method and then calling the <i>run</i> method:

In [17]:
results = survey.by(scenarios).by(agent).by(model).run()

## Inspecting the results
EDSL comes with built-in methods for analyzing results. For more details on working with results, please see the <a href="https://docs.expectedparrot.com/en/latest/results.html">Results</a> section of the docs.

To inspect the components of results:

In [18]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'agent.persona',
 'agent.years_experience',
 'answer.accident',
 'answer.issues',
 'answer.primary_issue',
 'answer.priority',
 'answer.refund',
 'answer.sentiment',
 'comment.accident_comment',
 'comment.issues_comment',
 'comment.priority_comment',
 'comment.refund_comment',
 'comment.sentiment_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.accident_system_prompt',
 'prompt.accident_user_prompt',
 'prompt.issues_system_prompt',
 'prompt.issues_user_prompt',
 'prompt.primary_issue_system_prompt',
 'prompt.primary_issue_user_prompt',
 'prompt.priority_system_prompt',
 'prompt.priority_user_prompt',
 'prompt.refund_system_prompt',
 'prompt.refund_user_prompt',
 'prompt.sentiment_system_prompt',
 'prompt.sentiment_user_prompt',
 'question_options.accident_question_options',

We can select just the responses to the questions to display:

In [19]:
results.select('ticket', 'issues', 'primary_issue', 'accident', 'sentiment', 'refund', 'priority').print()

scenario.ticket,answer.issues,answer.primary_issue,answer.accident,answer.sentiment,answer.refund,answer.priority
I just realized I left my phone in the car on my last ride. Can you help me get it back?,['lost item'],The primary issue is that the customer has left their phone in the car during their last ride and is seeking assistance to retrieve it.,Not applicable,Somewhat negative,No,4
I'm unhappy with my recent experience. The driver was very rude and unprofessional.,['driver performance'],The primary issue in this ticket is that the customer experienced rudeness and unprofessional behavior from a driver.,No,Very negative,No,4
I was charged more than the estimated fare for my trip yesterday. Can you explain why?,['other'],The primary issue in this ticket is that the customer was charged an amount exceeding the estimated fare for their trip and is seeking an explanation for the discrepancy.,Not applicable,Somewhat negative,No,3
"The car seat provided was not properly installed, and I felt my child was at risk. Please ensure driver training.","['safety', 'driver performance']","The primary issue is that the car seat provided was not properly installed, which led to concerns about the child's safety. The customer is requesting that drivers receive proper training on how to install car seats correctly.",No,Somewhat negative,No,5
"My driver took a longer route than necessary, resulting in a higher fare. I request a fare adjustment.","['driver performance', 'GPS/route']","The primary issue in the ticket is that the customer's driver took a longer route than necessary, which led to an increased fare. The customer is requesting a fare adjustment due to this inconvenience.",No,Somewhat negative,No,3
I had a great experience with my driver today! Very friendly and efficient service.,['driver performance'],There is no issue reported in this ticket. The customer is providing positive feedback about their great experience with the driver.,Not applicable,Very positive,No,0
I'm concerned about the vehicle's cleanliness. It was not up to the standard I expect.,['cleanliness'],"The primary issue in this ticket is the customer's dissatisfaction with the cleanliness of the vehicle, which did not meet their expected standard.",No,Somewhat negative,No,3
The app keeps crashing every time I try to book a ride. Please fix this issue.,['other'],The primary issue is that the app is crashing during the ride booking process.,Not applicable,Somewhat negative,No,4
"My driver was exceptional - safe driving, polite, and the car was spotless. Kudos!","['safety', 'cleanliness', 'driver performance']",There is no issue reported in this ticket. The customer is providing positive feedback about their exceptional experience with the driver.,No,Very positive,No,0
I felt unsafe during my ride due to the driver's erratic behavior. This needs to be addressed immediately.,"['safety', 'driver performance']",The primary issue in the ticket is that the customer felt unsafe due to the driver's erratic behavior during the ride.,No,Very negative,No,5


We can apply some lables to our table:

In [20]:
(results
 .select('ticket', 'issues', 'primary_issue', 'accident', 'sentiment', 'refund', 'priority')
 .print(pretty_labels={
     'scenario.ticket':'Ticket',
     'answer.issues':'Issues',
     'answer.primary_issue':'Primary issue',
     'answer.accident':'Accident',
     'answer.sentiment':'Sentiment',
     'answer.refund':'Refund request',
     'answer.priority':'Priority'})
)

Ticket,Issues,Primary issue,Accident,Sentiment,Refund request,Priority
I just realized I left my phone in the car on my last ride. Can you help me get it back?,['lost item'],The primary issue is that the customer has left their phone in the car during their last ride and is seeking assistance to retrieve it.,Not applicable,Somewhat negative,No,4
I'm unhappy with my recent experience. The driver was very rude and unprofessional.,['driver performance'],The primary issue in this ticket is that the customer experienced rudeness and unprofessional behavior from a driver.,No,Very negative,No,4
I was charged more than the estimated fare for my trip yesterday. Can you explain why?,['other'],The primary issue in this ticket is that the customer was charged an amount exceeding the estimated fare for their trip and is seeking an explanation for the discrepancy.,Not applicable,Somewhat negative,No,3
"The car seat provided was not properly installed, and I felt my child was at risk. Please ensure driver training.","['safety', 'driver performance']","The primary issue is that the car seat provided was not properly installed, which led to concerns about the child's safety. The customer is requesting that drivers receive proper training on how to install car seats correctly.",No,Somewhat negative,No,5
"My driver took a longer route than necessary, resulting in a higher fare. I request a fare adjustment.","['driver performance', 'GPS/route']","The primary issue in the ticket is that the customer's driver took a longer route than necessary, which led to an increased fare. The customer is requesting a fare adjustment due to this inconvenience.",No,Somewhat negative,No,3
I had a great experience with my driver today! Very friendly and efficient service.,['driver performance'],There is no issue reported in this ticket. The customer is providing positive feedback about their great experience with the driver.,Not applicable,Very positive,No,0
I'm concerned about the vehicle's cleanliness. It was not up to the standard I expect.,['cleanliness'],"The primary issue in this ticket is the customer's dissatisfaction with the cleanliness of the vehicle, which did not meet their expected standard.",No,Somewhat negative,No,3
The app keeps crashing every time I try to book a ride. Please fix this issue.,['other'],The primary issue is that the app is crashing during the ride booking process.,Not applicable,Somewhat negative,No,4
"My driver was exceptional - safe driving, polite, and the car was spotless. Kudos!","['safety', 'cleanliness', 'driver performance']",There is no issue reported in this ticket. The customer is providing positive feedback about their exceptional experience with the driver.,No,Very positive,No,0
I felt unsafe during my ride due to the driver's erratic behavior. This needs to be addressed immediately.,"['safety', 'driver performance']",The primary issue in the ticket is that the customer felt unsafe due to the driver's erratic behavior during the ride.,No,Very negative,No,5


EDSL also comes with methods for accessing results as a dataframe or SQL table:

In [21]:
df = results.to_pandas()
df

,agent.agent_instruction,agent.agent_name,agent.persona,agent.years_experience,answer.accident,answer.issues,answer.primary_issue,answer.priority,answer.refund,answer.sentiment,...,question_type.sentiment_question_type,raw_model_response.accident_raw_model_response,raw_model_response.issues_raw_model_response,raw_model_response.primary_issue_raw_model_response,raw_model_response.priority_raw_model_response,raw_model_response.refund_raw_model_response,raw_model_response.sentiment_raw_model_response,scenario.edsl_class_name,scenario.edsl_version,scenario.ticket
0,You are answering questions as if you were a h...,Agent_0,You are an expert customer service agent.,15,Not applicable,['lost item'],The primary issue is that the customer has lef...,4,No,Somewhat negative,...,multiple_choice,{'id': 'chatcmpl-9PuaIDVzpogKRTvred7xXgGLElAqn...,{'id': 'chatcmpl-9PuaIMwhphiISoyZIrdg7JeK5iwxk...,{'id': 'chatcmpl-9PuaIdoG3nuiXfY3ppOGXiN0V8uYV...,{'id': 'chatcmpl-9PuaI2S676vf1HFXBOEwoMkLamfm9...,{'id': 'chatcmpl-9PuaIgmAih5cbvg9P0P9MmUJlUUPx...,{'id': 'chatcmpl-9PuaIwQ4yDeHbCncrSfecKsjQJYCg...,Scenario,0.1.23,I just realized I left my phone in the car on ...
1,You are answering questions as if you were a h...,Agent_0,You are an expert customer service agent.,15,No,['driver performance'],The primary issue in this ticket is that the c...,4,No,Very negative,...,multiple_choice,{'id': 'chatcmpl-9PuaIw4uQEqZKstoPGvb5hJh3OAnV...,{'id': 'chatcmpl-9PuaIQHe3mPj1NHhIiDyZl2F1TBdY...,{'id': 'chatcmpl-9PuaISZkTYLiAYqM5hgXCsVuunIVy...,{'id': 'chatcmpl-9PuaIops5rP6vdZSXjsca0u93iEqs...,{'id': 'chatcmpl-9PuaIZ1ZpO4k9xf4WuXI4NQ8F28hV...,{'id': 'chatcmpl-9PuaIpYEYhp6mWGvZKqnJQfuIxWiN...,Scenario,0.1.23,I'm unhappy with my recent experience. The dri...
2,You are answering questions as if you were a h...,Agent_0,You are an expert customer service agent.,15,Not applicable,['other'],The primary issue in this ticket is that the c...,3,No,Somewhat negative,...,multiple_choice,{'id': 'chatcmpl-9PuaIvrkrgQioEq9ckQphh3sF5hst...,{'id': 'chatcmpl-9PuaIIGvi2Te62Wun7iTzZWbWbTxB...,{'id': 'chatcmpl-9PuaIeNQPbWqNi6TsXLrwAUlzcXqM...,{'id': 'chatcmpl-9PuaINiBjHonmuO0HMOGgXRfoEas2...,{'id': 'chatcmpl-9PuaIUU15ZMQfSsIvAqWl8VUv6JUv...,{'id': 'chatcmpl-9PuaIItdwNGolYqNndxzUrejGpklv...,Scenario,0.1.23,I was charged more than the estimated fare for...
3,You are answering questions as if you were a h...,Agent_0,You are an expert customer service agent.,15,No,"['safety', 'driver performance']",The primary issue is that the car seat provide...,5,No,Somewhat negative,...,multiple_choice,{'id': 'chatcmpl-9PuaI7Y7ySWH8VhxOOvuux2ywwz5v...,{'id': 'chatcmpl-9PuaIRsav26gGPVM75DAtyZJqD9T9...,{'id': 'chatcmpl-9PuaInEkqKtQCGLZHwayXVvQDQ1aU...,{'id': 'chatcmpl-9PuaIOQisWiaw71LsD3FtrE3U7QC4...,{'id': 'chatcmpl-9PuaIqddYIvmkQa04lknJ9CAyWZWD...,{'id': 'chatcmpl-9PuaIX3MMrGIk5DUHd8qoorNOqFUI...,Scenario,0.1.23,The car seat provided was not properly install...
4,You are answering questions as if you were a h...,Agent_0,You are an expert customer service agent.,15,No,"['driver performance', 'GPS/route']",The primary issue in the ticket is that the cu...,3,No,Somewhat negative,...,multiple_choice,{'id': 'chatcmpl-9PuaICFuvncAkjriLxw2mezX1z0wj...,{'id': 'chatcmpl-9PuaI6GV47gMi6Q5v1u2ESnr2heqw...,{'id': 'chatcmpl-9PuaImGt2oMLpQU4UFrfgQcDd56Fm...,{'id': 'chatcmpl-9PuaIj7eHa86GS0lspcy9NY9ZGM21...,{'id': 'chatcmpl-9PuaIDgN1iAAnlHfq0bgviKFXkmQk...,{'id': 'chatcmpl-9PuaIgvJ97JjWD91C0NM6wnzDkdQO...,Scenario,0.1.23,"My driver took a longer route than necessary, ..."
5,You are answering questions as if you were a h...,Agent_0,You are an expert customer service agent.,15,Not applicable,['driver performance'],There is no issue reported in this ticket. The...,0,No,Very positive,...,multiple_choice,{'id': 'chatcmpl-9PuaIAE4yQWe9Q45hJviLHgqBQyPf...,{'id': 'chatcmpl-9PuaIFwSRgkO3VQQRvhfPi5hUhppj...,{'id': 'chatcmpl-9PuaI3OR6P9vDJZGz9C2i6CcmXnPu...,{'id': 'chatcmpl-9PuaITxsKnfMA15FlHu16MOWZx7Rh...,{'id': 'chatcmpl-9PuaIlZY

We can also access results as a SQL table:

In [22]:
results.sql("select * from self", shape="wide")

,accident,accident_comment,accident_question_options,accident_question_text,accident_question_type,accident_raw_model_response,accident_system_prompt,accident_user_prompt,agent_instruction,agent_name,...,sentiment_question_text,sentiment_question_type,sentiment_raw_model_response,sentiment_system_prompt,sentiment_user_prompt,temperature,ticket,top_logprobs,top_p,years_experience
0,Not applicable,"The ticket is regarding a lost phone, which is...","['Yes', 'No', 'Not applicable']","If the primary issue in this ticket is safety,...",multiple_choice,{'id': 'chatcmpl-9PuaIDVzpogKRTvred7xXgGLElAqn...,You are answering questions as if you were a h...,You are being asked the following question: If...,You are answering questions as if you were a h...,Agent_0,...,What is the sentiment of this ticket? Ticket: ...,multiple_choice,{'id': 'chatcmpl-9PuaIwQ4yDeHbCncrSfecKsjQJYCg...,You are answering questions as if you were a h...,You are being asked the following question: Wh...,0.5,I just realized I left my phone in the car on ...,3,1,15
1,No,The ticket mentions an issue with the driver b...,"['Yes', 'No', 'Not applicable']","If the primary issue in this ticket is safety,...",multiple_choice,{'id': 'chatcmpl-9PuaIw4uQEqZKstoPGvb5hJh3OAnV...,You are answering questions as if you were a h...,You are being asked the following question: If...,You are answering questions as if you were a h...,Agent_0,...,What is the sentiment of this ticket? Ticket: ...,multiple_choice,{'id': 'chatcmpl-9PuaIpYEYhp6mWGvZKqnJQfuIxWiN...,You are answering questions as if you were a h...,You are being asked the following question: Wh...,0.5,I'm unhappy with my recent experience. The dri...,3,1,15
2,Not applicable,"The ticket is regarding a fare discrepancy, no...","['Yes', 'No', 'Not applicable']","If the primary issue in this ticket is safety,...",multiple_choice,{'id': 'chatcmpl-9PuaIvrkrgQioEq9ckQphh3sF5hst...,You are answering questions as if you were a h...,You are being asked the following question: If...,You are answering questions as if you were a h...,Agent_0,...,What is the sentiment of this ticket? Ticket: ...,multiple_choice,{'id': 'chatcmpl-9PuaIItdwNGolYqNndxzUrejGpklv...,You are answering questions as if you were a h...,You are being asked the following question: Wh...,0.5,I was charged more than the estimated fare for...,3,1,15
3,No,"No accident was reported, only a concern for c...","['Yes', 'No', 'Not applicable']","If the primary issue in this ticket is safety,...",multiple_choice,{'id': 'chatcmpl-9PuaI7Y7ySWH8VhxOOvuux2ywwz5v...,You are answering questions as if you were a h...,You are being asked the following question: If...,You are answering questions as if you were a h...,Agent_0,...,What is the sentiment of this ticket? Ticket: ...,multiple_choice,{'id': 'chatcmpl-9PuaIX3MMrGIk5DUHd8qoorNOqFUI...,You are answering questions as if you were a h...,You are being asked the following question: Wh...,0.5,The car seat provided was not properly install...,3,1,15
4,No,The primary issue mentioned in the ticket does...,"['Yes', 'No', 'Not applicable']","If the primary issue in this ticket is safety,...",multiple_choice,{'id': 'chatcmpl-9PuaICFuvncAkjriLxw2mezX1z0wj...,You are answering questions as if you were a h...,You are being asked the following question: If...,You are answering questions as if you were a h...,Agent_0,...,What is the sentiment of this ticket? Ticket: ...,multiple_choice,{'id': 'chatcmpl-9PuaIgvJ97JjWD91C0NM6wnzDkdQO...,You are answering questions as if you were a h...,You are being asked the following question: Wh...,0.5,"My driver took a longer route than necessary, ...",3,1,15
5,Not applicable,The ticket mentions a positive experience with...,"['Yes', 'No', 'Not applicable']","If the primary issue in this ticket is safety,...",multiple_choice,{'id': 'chatcmpl-9PuaIAE4yQWe9Q45hJviLHgqBQyPf...,You are answering questions as if you were a h...,You are being asked the following question: If...,You are answering questions as if you

To export results to a CSV file:

In [23]:
results.to_csv("data_labeling_example.csv")